In [ ]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random

In [ ]:
# this function creates a hourly pattern of multipliers (up to 26280 hours long) mixing together n (max 1950) patterns
# from the STREaM dataset; it scales within a range (as expected for WNTR/EPANET input)
def create_demand_pattern(df_patterns, n_hours=168, n_mix=50, bound = [0.3,0.9], noise_intensity=0.01):
    cols =df_patterns.columns.to_list()
    shuffle(cols)
    df_patterns = df_patterns[cols]
    s = MinMaxScaler(bound)
    temp=np.squeeze(s.fit_transform(df_patterns.iloc[:n_hours,:n_mix].mean(axis=1).values.reshape(-1,1)))
    dem_pat = temp + np.random.randn(n_hours)*noise_intensity
    return dem_pat

In [ ]:
# load STREaM data (STREaM is a simulator of end-uses and water demand patterns)
# This dataset contains 3 years of hourly (26280 data points) synthetic demand data 
# for 4 user_classes = ['low','average','high','morning','night']. There are 500 user for each class,
# with the first 10 users having leaks. We need to remove these users for our purposes. 

all_data_path = './data/allData.txt'
df = pd.read_csv(all_data_path, header=None)
# 
# create headers
user_classes = ['low','average','high','morning','night']
headers = []
for user_class in user_classes:
    suffixes = (('_'+user_class+' ')*400).split() # original
    for suffix, number in zip(suffixes,range(1,401)):
        headers.append('user'+suffix+'_'+'%03d'%number)  
        
# reverse df and change df columns
df = df.T
df.columns = headers

# remove first 10 of each class (these have leaks, so not useful for our purposes)
users_to_remove = np.array(headers).reshape(-1,10)[::40].ravel().tolist()
df.drop(users_to_remove, axis = 1, inplace = True)
headers = df.columns

In [ ]:
# load EPANET model
inp_file = 'ctown_map_with_controls.inp'
out_file_pattern = 'second_batch'
topo_name = 'ctown'
wn = wntr.network.WaterNetworkModel(inp_file)

# save original patterns
names, patterns =[],[]
for name, pat in wn.patterns():
    names.append(name)
    patterns.append(pat.multipliers)
df_pat_orig = pd.DataFrame(data = patterns).T
df_pat_orig.columns = names

In [ ]:
# Generate a 3-year pattern for the District Metered Area (DMA)
names, patterns =[],[]
for name, pat in wn.patterns():
    names.append(name)
    temp = create_demand_pattern(df, n_hours=24*365*3, n_mix=150)
    pat.multipliers = temp
    patterns.append(pat.multipliers)
df_pat_new = pd.DataFrame(data = patterns).T
df_pat_new.columns = names
output_total_demands_path = out_file_pattern + '_' + topo_name + '_three_year_demands.csv'
df_pat_new.to_csv(output_total_demands_path)

In [ ]:
i=1
for pat_name in names:
    dd = plt.subplot(len(names),1,i)
    plt.plot(df_pat_new[pat_name], label="Demand")
    plt.title(pat_name)
    plt.subplots_adjust(top=2.00, right=3.00)
    plt.xlim(left=0, right=1000)
    plt.grid(True)
    i+=1

In [ ]:
# Create a pandas dataframe with columns = each DMA pattern and 38 random initial pattern values
df_pat_new = pd.read_csv(output_total_demands_path)
values=[]
weekly_simulation_duration = 168
ten_days_simulatino_duration = 240
weekly=False

if weekly:
    limit = weekly_simulation_duration
else:
    limit = ten_days_simulatino_duration

for i in range(0,38):
    value = random.randint(0, len(df_pat_new)-limit)        
    values.append(value)
pattern_random_init_point = pd.DataFrame(data = values)
pattern_random_init_point.columns = ['start']
print(pattern_random_init_point)
initial_points_patch = out_file_pattern + '_' + topo_name + '_demand_starting_points.csv'
pattern_random_init_point.to_csv(initial_points_patch)

In [ ]:
# Create a pandas dataframe with initial tank levels for the 7 C-Town tanks and 38 weeks
values=[]
tank_names = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']
#tank_names = ['TANK']
for name in tank_names:
    aux=[]
    for i in range(0,38):
        tank_max_value = wn.get_node(name).max_level
        min_value = tank_max_value*0.10
        max_value = tank_max_value*0.80
        value = random.uniform(min_value, max_value)        
        aux.append(value)
    values.append(aux)
tank_initial_levels = pd.DataFrame(data = values).T
tank_initial_levels.columns = tank_names
print(tank_initial_levels)
tank_levels_patch = out_file_pattern + '_' + topo_name + '_tank_initial_conditions.csv'
tank_initial_levels.to_csv(tank_levels_patch)